In [1]:
pip install scapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444345 sha256=ec4a948c2d1171c4320768c24ef28a1af4b026dea6b3233a0e3475eab5662aa0
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [2]:
from scapy.all import *
def capture_packets():
    # Capture network packets using Scapy
    packets = sniff(filter="ip", count=100)
    return packets

In [3]:
packets = capture_packets()
print(packets)

ERROR: Cannot set filter: libpcap is not available. Cannot compile filter !
ERROR:scapy.runtime:Cannot set filter: libpcap is not available. Cannot compile filter !


<Sniffed: TCP:100 UDP:0 ICMP:0 Other:0>


In [4]:
def extract_features(packets):
    features = []
    for packet in packets:
        if IP in packet:
            ip_src = packet[IP].src
            ip_dst = packet[IP].dst
            ip_len = packet[IP].len
            ip_proto = packet[IP].proto
        else:
            continue
        if TCP in packet:
            tcp_sport = packet[TCP].sport
            tcp_dport = packet[TCP].dport
            tcp_flags = packet[TCP].flags
        else:
            tcp_sport = -1
            tcp_dport = -1
            tcp_flags = -1
        if UDP in packet:
            udp_sport = packet[UDP].sport
            udp_dport = packet[UDP].dport
        else:
            udp_sport = -1
            udp_dport = -1
        features.append([ip_src, ip_dst, ip_len, ip_proto, tcp_sport, tcp_dport, tcp_flags, udp_sport, udp_dport])
    return features

In [5]:
packets = capture_packets()
features = extract_features(packets)

ERROR: Cannot set filter: libpcap is not available. Cannot compile filter !
ERROR:scapy.runtime:Cannot set filter: libpcap is not available. Cannot compile filter !


In [6]:
def extract_features(packet):
    features = {}
    # Extract source and destination IP addresses
    features['src_ip'] = packet[IP].src
    features['dst_ip'] = packet[IP].dst
    # Extract port numbers if the packet has a TCP layer
    if packet.haslayer(TCP):
        features['src_port'] = packet[TCP].sport
        features['dst_port'] = packet[TCP].dport
    # Extract packet length
    features['length'] = len(packet)
    return features

In [7]:
features = []
for packet in packets:
    features.append(extract_features(packet))

In [8]:
import pandas as pd

# Convert extracted features to a dataframe
df = pd.DataFrame(features)

# Save dataframe to CSV file
df.to_csv('captured_traffic.csv', index=False)

In [9]:
df

,src_ip,dst_ip,src_port,dst_port,length
0,172.28.0.1,172.28.0.12,39934,8080,960
1,172.28.0.12,172.28.0.1,8080,39934,66
2,172.28.0.12,172.28.0.1,8080,39934,826
3,172.28.0.1,172.28.0.12,39934,8080,66
4,172.28.0.1,172.28.0.12,39934,8080,960
...,...,...,...,...,...
95,172.28.0.1,172.28.0.12,54576,8080,66
96,172.28.0.1,172.28.0.12,54576,8080,960
97,172.28.0.1,172.28.0.12,53680,8080,1065
98,172.28.0.12,172.28.0.1,8080,53680,66


In [10]:
def detect_attack(packet):
    """
    Determine the type of attack based on the packet.
    """
    # Check if the packet is a SYN Flood attack
    if TCP in packet and packet[TCP].flags == "S":
        return "SYN_Flood"
    # Check if the packet is a UDP Flood attack
    elif UDP in packet:
        return "UDP_Flood"
    # Otherwise, assume normal traffic
    else:
        return "Normal"

In [11]:
from sklearn.model_selection import train_test_split

# Define the target variable
labels = [detect_attack(packet) for packet in packets]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

In [12]:
import ipaddress

def ip_to_int(ip_address):
    return int(ipaddress.IPv4Address(ip_address))

# Convert IP addresses to integers
print(X_train)
print(y_train)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train['src_ip'] = X_train['src_ip'].apply(ip_to_int)
X_test['src_ip'] = X_test['src_ip'].apply(ip_to_int)
X_train['dst_ip'] = X_train['dst_ip'].apply(ip_to_int)
X_test['dst_ip'] = X_test['dst_ip'].apply(ip_to_int)

[{'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 39934, 'dst_port': 8080, 'length': 66}, {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 39934, 'length': 388}, {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 53680, 'length': 388}, {'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 39934, 'dst_port': 8080, 'length': 66}, {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 53680, 'length': 388}, {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 39934, 'length': 1246}, {'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 54576, 'dst_port': 8080, 'length': 66}, {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 54576, 'length': 351}, {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 53680, 'length': 66}, {'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 39934, 'dst_p

In [13]:
from inspect import modulesbyfile
# Import the LogisticRegression class
from sklearn.linear_model import LogisticRegression

# Create an instance of the LogisticRegression class
model = LogisticRegression()

# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)



In [14]:
# Evaluate the performance of the model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


In [15]:
from scapy.all import *
import numpy as np

# Initialize the list of packets
packets = []

def extract_features(packet):
    """
    Extract relevant features from a packet.
    """
    features = {}
    # Check if the packet contains an IP layer
    if IP in packet:
        # Extract source and destination IP addresses
        features['src_ip'] = packet[IP].src
        features['dst_ip'] = packet[IP].dst
        # Check if the packet contains a TCP layer
        if TCP in packet:
            # Extract port numbers
            features['src_port'] = packet[TCP].sport
            features['dst_port'] = packet[TCP].dport
            # Extract the TCP flags
            features['flags'] = packet[TCP].flags
        # Check if the packet contains a UDP layer
        elif UDP in packet:
            # Extract port numbers
            features['src_port'] = packet[UDP].sport
            features['dst_port'] = packet[UDP].dport
        # Check if the packet contains an ICMP layer
        elif ICMP in packet:
            # Extract the ICMP type and code
            features['type'] = packet[ICMP].type
            features['code'] = packet[ICMP].code
    return features

def handle_packet(packet):
    """
    Callback function for processing packets.
    """
    # Extract features from the packet
    features = extract_features(packet)
    # Append the features to the list of packets
    packets.append(features)

# Start the packet sniffer
sniff(prn=handle_packet, count=100)

# Convert the list of packets to a numpy array
data = np.array(packets)

# Print the data
print(data)


[{'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 33532, 'dst_port': 8080, 'flags': <Flag 24 (PA)>}
 {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 33532, 'flags': <Flag 16 (A)>}
 {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 33532, 'flags': <Flag 24 (PA)>}
 {'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 33532, 'dst_port': 8080, 'flags': <Flag 16 (A)>}
 {'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 33532, 'dst_port': 8080, 'flags': <Flag 24 (PA)>}
 {'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 33532, 'dst_port': 8080, 'flags': <Flag 24 (PA)>}
 {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 33532, 'flags': <Flag 16 (A)>}
 {'src_ip': '172.28.0.12', 'dst_ip': '172.28.0.1', 'src_port': 8080, 'dst_port': 33532, 'flags': <Flag 24 (PA)>}
 {'src_ip': '172.28.0.1', 'dst_ip': '172.28.0.12', 'src_port': 33532, 'dst_port': 8080, 'flags': <F

# New Section

# New Section